In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb
import quimb.tensor as qtn
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from functions import *

dimension of dataset: 14

In [ ]:
def MMD(x, y,Ommd, sigma, number_open_index, bond_dimension):
    """
    samples and target are two Matrix Product States.
    """

    x /= x.H @ x
    y /= y.H @ y
    rename_dict = {f'k{i}': f'k{i+number_open_index}' for i in range(number_open_index)}
    y.reindex_(rename_dict)

    """
    Building the MMD MPO. The default open indexes are k1, k2, ..., kn, b1, b2, ..., bn.
    Then we can contract the MPO with the MPS and the bitstring state to get the loss function.
    """
    loss = x & Ommd & y
    # rename_dict = {f'k{i}': f'b{i}' for i in range(number_open_index)}
    # x.reindex_(rename_dict)
    # loss = loss & x
    # rename_dict = {f'k{i}': f'b{i+number_open_index}' for i in range(number_open_index)}
    # y.reindex_(rename_dict)
    # loss = loss & x.H & y.H
    # loss = loss^...

    return loss

In [3]:
L = 9
D = 8
sigma = 0.09

# create a random MPS as our initial target to optimize
psi = qtn.MPS_rand_state(L, bond_dim=D)
Ommd = Ommd(L, sigma)
dataset = get_bars_and_stripes(3)

In [ ]:
MPS_dataset = []
for data in dataset:
    MPS_dataset.append(qtn.MPS_computational_state(data))

In [7]:
def loss_fn(psi,dataset,mpo):
    loss = 0
    for data in dataset:
        data_copy = data.copy()
        psi_copy = psi.copy()
        loss += MMD(psi, psi_copy, mpo, sigma, L, D) - 2*MMD(psi, data, mpo, sigma, L, D) + MMD(data_copy, data, mpo, sigma, L, D)
        loss = loss / len(dataset)
    return loss


In [42]:
tnopt = qtn.TNOptimizer(
    # the tensor network we want to optimize
    psi,
    # the functions specfying the loss and normalization
    loss_fn=loss_fn,
    #norm_fn=norm_fn,
    # we specify constants so that the arguments can be converted
    # to the  desired autodiff backend automatically
    loss_constants={"dataset": MPS_dataset, "mpo": Ommd},
    # the underlying algorithm to use for the optimization
    # 'l-bfgs-b' is the default and often good for fast initial progress
    optimizer="L-BFGS-B",
    # which gradient computation backend to use
    #autodiff_backend="autograd",
)

In [ ]:
#psi_opt = tnopt.optimize(5)

-4863669442333533006651827358497123129697291620473529419032704520728208059320121209960939858214304519247836718960009182944658230013459735306535825007344070828072316103654528516096.000000000000 [best: -4863669442333533006651827358497123129697291620473529419032704520728208059320121209960939858214304519247836718960009182944658230013459735306535825007344070828072316103654528516096.000000000000] : : 20it [03:01,  9.07s/it]


In [22]:
    del loss

NameError: name 'loss' is not defined

In [21]:
MMD(psi,psi.copy(),Ommd,sigma,L,D).draw(figsize=(30, 30), show_inds=True)

ValueError: The index b0 appears more than twice! If this is intentionally a 'hyper' tensor network you will need to explicitly supply `output_inds` when contracting for example.

In [8]:
# test
import time
L = 9
D = 8
sigma = 0.09
dataset = get_bars_and_stripes(3)
for i in range(10):
    psi = qtn.MPS_rand_state(L, bond_dim=D, seed=i**i)
    print(MMD(psi,psi.copy(),Ommd,sigma,L,D)+MMD(MPS_dataset[i],MPS_dataset[i].copy(),Ommd,sigma,L,D)-2*MMD(psi,MPS_dataset[i],Ommd,sigma,L,D))

NameError: name 'MPS_dataset' is not defined